# इंटेंट-आधारित मॉडेल राउटर फाउंड्री लोकल SDK सह

**CPU-ऑप्टिमाइझ्ड मल्टी-मॉडेल राउटिंग सिस्टम**

या नोटबुकमध्ये एक बुद्धिमान राउटिंग सिस्टम दाखवली आहे जी वापरकर्त्याच्या हेतूनुसार सर्वोत्तम लहान भाषा मॉडेल आपोआप निवडते. अशा एज डिप्लॉयमेंट परिस्थितीसाठी योग्य आहे जिथे तुम्ही अनेक विशेष मॉडेल्स कार्यक्षमतेने वापरू इच्छिता.

## 🎯 तुम्ही काय शिकाल

- **हेतू ओळखणे**: प्रॉम्प्ट्सचे आपोआप वर्गीकरण करा (कोड, सारांश, वर्गीकरण, सामान्य)
- **स्मार्ट मॉडेल निवड**: प्रत्येक कार्यासाठी सर्वात सक्षम मॉडेलकडे रूट करा
- **CPU ऑप्टिमायझेशन**: कोणत्याही हार्डवेअरवर कार्य करणारी मेमरी-कार्यक्षम मॉडेल्स
- **मल्टी-मॉडेल व्यवस्थापन**: `--retain true` सह अनेक मॉडेल्स लोड ठेवा
- **उत्पादन पॅटर्न्स**: रीट्राय लॉजिक, एरर हँडलिंग, आणि टोकन ट्रॅकिंग

## 📋 परिस्थितीचा आढावा

या पॅटर्नमध्ये दाखवले आहे:

1. **हेतू ओळखणे**: प्रत्येक वापरकर्त्याच्या प्रॉम्प्टचे वर्गीकरण करा (कोड, सारांश, वर्गीकरण, किंवा सामान्य)
2. **मॉडेल निवड**: क्षमतांवर आधारित सर्वात योग्य लहान भाषा मॉडेल आपोआप निवडा
3. **स्थानिक अंमलबजावणी**: फाउंड्री लोकल सेवेद्वारे स्थानिक मॉडेल्सकडे रूट करा
4. **एकसंध इंटरफेस**: अनेक विशेष मॉडेल्सकडे रूट करणारे एकच चॅट प्रवेश बिंदू

**योग्य आहे**: अनेक विशेष मॉडेल्ससह एज डिप्लॉयमेंट्ससाठी जिथे तुम्हाला मॅन्युअल मॉडेल निवडीशिवाय बुद्धिमान विनंती राउटिंग हवे आहे.

## 🔧 पूर्वतयारी

- **Foundry Local** स्थापित आणि सेवा चालू
- **Python 3.8+** pip सह
- **8GB+ RAM** (अनेक मॉडेल्ससाठी 16GB+ शिफारस केली जाते)
- **workshop_utils** मॉड्यूल (../samples/ मध्ये)

## 🚀 जलद सुरुवात

नोटबुक:
1. तुमच्या सिस्टम मेमरीचा शोध घेईल
2. योग्य CPU मॉडेल्सची शिफारस करेल
3. `--retain true` सह मॉडेल्स आपोआप लोड करेल
4. सर्व मॉडेल्स तयार असल्याची पडताळणी करेल
5. चाचणी प्रॉम्प्ट्स विशेष मॉडेल्सकडे रूट करेल

**अंदाजे सेटअप वेळ**: 5-7 मिनिटे (मॉडेल लोडिंगसह)


## 📦 चरण 1: अवलंबित्वे स्थापित करा

अधिकृत Foundry Local SDK आणि आवश्यक लायब्ररी स्थापित करा:

- **foundry-local-sdk**: स्थानिक मॉडेल व्यवस्थापनासाठी अधिकृत Python SDK  
- **openai**: चॅट पूर्णतेसाठी OpenAI-सुसंगत API  
- **psutil**: प्रणाली मेमरी शोध आणि निरीक्षणासाठी  


In [107]:
# Install core dependencies
!pip install -q foundry-local-sdk openai psutil

## 💻 चरण 2: सिस्टम मेमरी शोधा

उपलब्ध सिस्टम मेमरी शोधा जेणेकरून कोणते CPU मॉडेल्स कार्यक्षमतेने चालवता येतील हे ठरवता येईल. यामुळे तुमच्या हार्डवेअरसाठी सर्वोत्तम मॉडेल निवड सुनिश्चित होते.


In [108]:
import psutil

# Get system memory information
total_memory_gb = psutil.virtual_memory().total / (1024**3)
available_memory_gb = psutil.virtual_memory().available / (1024**3)

print('🖥️  System Memory Information')
print('=' * 70)
print(f'Total Memory:     {total_memory_gb:.2f} GB')
print(f'Available Memory: {available_memory_gb:.2f} GB')
print()

# Recommend models based on available memory
# Using model aliases - Foundry Local will automatically select CPU variant
model_aliases = []

if total_memory_gb >= 32:
    model_aliases = ['phi-4-mini', 'phi-3.5-mini', 'qwen2.5-0.5b', 'qwen2.5-coder-0.5b']
    print('✅ High Memory System (32GB+)')
    print('   Can run 3-4 models simultaneously')
elif total_memory_gb >= 16:
    model_aliases = ['phi-4-mini', 'qwen2.5-0.5b', 'phi-3.5-mini']
    print('✅ Medium Memory System (16-32GB)')
    print('   Can run 2-3 models simultaneously')
elif total_memory_gb >= 8:
    model_aliases = ['qwen2.5-0.5b', 'phi-3.5-mini']
    print('⚠️  Lower Memory System (8-16GB)')
    print('   Recommended: 2 smaller models')
else:
    model_aliases = ['qwen2.5-0.5b']
    print('⚠️  Limited Memory System (<8GB)')
    print('   Recommended: Use only smallest model')

print()
print('📋 Recommended Model Aliases for Your System:')
for model in model_aliases:
    print(f'   • {model}')

print()
print('💡 About Model Aliases:')
print('   ✓ Use base alias (e.g., phi-4-mini, not phi-4-mini-cpu)')
print('   ✓ Foundry Local automatically selects CPU variant for your hardware')
print('   ✓ No GPU required - optimized for CPU inference')
print('   ✓ Predictable memory usage and consistent performance')
print('=' * 70)

🖥️  System Memory Information
Total Memory:     63.30 GB
Available Memory: 16.19 GB

✅ High Memory System (32GB+)
   Can run 3-4 models simultaneously

📋 Recommended Model Aliases for Your System:
   • phi-4-mini
   • phi-3.5-mini
   • qwen2.5-0.5b
   • qwen2.5-coder-0.5b

💡 About Model Aliases:
   ✓ Use base alias (e.g., phi-4-mini, not phi-4-mini-cpu)
   ✓ Foundry Local automatically selects CPU variant for your hardware
   ✓ No GPU required - optimized for CPU inference
   ✓ Predictable memory usage and consistent performance


## 🤖 चरण 3: स्वयंचलित मॉडेल लोडिंग

या सेलद्वारे स्वयंचलितपणे:
1. Foundry Local सेवा सुरू केली जाते (जर ती चालू नसेल)
2. `--retain true` वापरून शिफारस केलेले मॉडेल्स लोड केले जातात (एकाच वेळी अनेक मॉडेल्स मेमरीत ठेवली जातात)
3. SDK वापरून सर्व मॉडेल्स तयार असल्याची खात्री केली जाते

⏱️ **अपेक्षित वेळ**: सर्व मॉडेल्ससाठी 3-5 मिनिटे


In [109]:
import subprocess
import time
import sys
import os

# Add samples directory for workshop_utils (Foundry SDK pattern)
sys.path.append(os.path.join('..', 'samples'))

print('🚀 Automatic Model Loading with SDK Verification')
print('=' * 70)

# Use top 3 recommended models (aliases)
# Foundry will automatically load CPU variants
REQUIRED_MODELS = model_aliases[:3]
print(f'📋 Loading {len(REQUIRED_MODELS)} models: {REQUIRED_MODELS}')
print('💡 Using model aliases - Foundry will load CPU variants automatically')
print()

# Step 1: Ensure Foundry Local service is running
print('📡 Step 1: Checking Foundry Local service...')
try:
    result = subprocess.run(['foundry', 'service', 'status'], 
                          capture_output=True, text=True, timeout=5)
    if result.returncode == 0:
        print('   ✅ Service is already running')
    else:
        print('   ⚙️  Starting Foundry Local service...')
        subprocess.run(['foundry', 'service', 'start'], 
                      capture_output=True, text=True, timeout=30)
        time.sleep(5)
        print('   ✅ Service started')
except Exception as e:
    print(f'   ⚠️  Could not verify service: {e}')
    print('   💡 Try manually: foundry service start')

# Step 2: Load each model with --retain true
print(f'\n🤖 Step 2: Loading models with retention...')
for i, model in enumerate(REQUIRED_MODELS, 1):
    print(f'   [{i}/{len(REQUIRED_MODELS)}] Starting {model}...')
    try:
        subprocess.Popen(['foundry', 'model', 'run', model, '--retain', 'true'],
                        stdout=subprocess.DEVNULL,
                        stderr=subprocess.DEVNULL)
        print(f'       ✅ {model} loading in background')
    except Exception as e:
        print(f'       ❌ Error starting {model}: {e}')

# Step 3: Verify models are ready
print(f'\n✅ Step 3: Verifying models (this may take 2-3 minutes)...')
print('=' * 70)

try:
    from workshop_utils import get_client
    
    ready_models = []
    max_attempts = 30
    attempt = 0
    
    while len(ready_models) < len(REQUIRED_MODELS) and attempt < max_attempts:
        attempt += 1
        print(f'\n   Attempt {attempt}/{max_attempts}...')
        
        for model in REQUIRED_MODELS:
            if model in ready_models:
                continue
                
            try:
                manager, client, model_id = get_client(model, None)
                response = client.chat.completions.create(
                    model=model_id,
                    messages=[{"role": "user", "content": "test"}],
                    max_tokens=5,
                    temperature=0
                )
                
                if response and response.choices:
                    ready_models.append(model)
                    print(f'   ✅ {model} is READY')
                    
            except Exception as e:
                error_msg = str(e).lower()
                if 'connection' in error_msg or 'timeout' in error_msg:
                    print(f'   ⏳ {model} still loading...')
                else:
                    print(f'   ⚠️  {model} error: {str(e)[:60]}...')
        
        if len(ready_models) == len(REQUIRED_MODELS):
            break
            
        if len(ready_models) < len(REQUIRED_MODELS):
            time.sleep(10)
    
    # Final status
    print('\n' + '=' * 70)
    print(f'📦 Final Status: {len(ready_models)}/{len(REQUIRED_MODELS)} models ready')
    
    for model in REQUIRED_MODELS:
        if model in ready_models:
            print(f'   ✅ {model} - READY (retained in memory)')
        else:
            print(f'   ❌ {model} - NOT READY')
    
    if len(ready_models) == len(REQUIRED_MODELS):
        print('\n🎉 All models loaded and verified!')
        print('   ✅ Ready for intent-based routing')
    else:
        print(f'\n⚠️  Some models not ready. Check: foundry model ls')
        
except ImportError as e:
    print(f'\n❌ Cannot import workshop_utils: {e}')
    print('   💡 Ensure workshop_utils.py is in ../samples/')
except Exception as e:
    print(f'\n❌ Verification error: {e}')

🚀 Automatic Model Loading with SDK Verification
📋 Loading 3 models: ['phi-4-mini', 'phi-3.5-mini', 'qwen2.5-0.5b']
💡 Using model aliases - Foundry will load CPU variants automatically

📡 Step 1: Checking Foundry Local service...
   ✅ Service is already running

🤖 Step 2: Loading models with retention...
   [1/3] Starting phi-4-mini...
       ✅ phi-4-mini loading in background
   [2/3] Starting phi-3.5-mini...
       ✅ phi-3.5-mini loading in background
   [3/3] Starting qwen2.5-0.5b...
       ✅ qwen2.5-0.5b loading in background

✅ Step 3: Verifying models (this may take 2-3 minutes)...

   Attempt 1/30...
   ⚠️  phi-4-mini error: get_client() takes 1 positional argument but 2 were given...
   ⚠️  phi-3.5-mini error: get_client() takes 1 positional argument but 2 were given...
   ⚠️  qwen2.5-0.5b error: get_client() takes 1 positional argument but 2 were given...

   Attempt 2/30...
   ⚠️  phi-4-mini error: get_client() takes 1 positional argument but 2 were given...
   ⚠️  phi-3.5-min

## 🎯 चरण 4: हेतू शोध आणि मॉडेल कॅटलॉग कॉन्फिगर करा

राउटिंग सिस्टम सेटअप करा:
- **हेतू नियम**: प्रॉम्प्ट वर्गीकरणासाठी Regex पॅटर्न्स
- **मॉडेल कॅटलॉग**: मॉडेल क्षमतांना हेतू श्रेणींशी जुळवते
- **प्राधान्य प्रणाली**: जेव्हा अनेक मॉडेल्स जुळतात तेव्हा मॉडेल निवड निश्चित करते

**CPU मॉडेलचे फायदे**:
- ✅ GPU आवश्यक नाही
- ✅ सातत्यपूर्ण कार्यक्षमता
- ✅ कमी ऊर्जा वापर
- ✅ अंदाजे मेमरी वापर


In [110]:
import re

# Model capability catalog (maps model aliases to capabilities)
# Use base aliases - Foundry Local will automatically select CPU variants
CATALOG = {
    'phi-4-mini': {
        'capabilities': ['general', 'summarize', 'reasoning'],
        'priority': 3
    },
    'qwen2.5-0.5b': {
        'capabilities': ['classification', 'fast', 'general'],
        'priority': 1
    },
    'phi-3.5-mini': {
        'capabilities': ['code', 'refactor', 'technical'],
        'priority': 2
    },
    'qwen2.5-coder-0.5b': {
        'capabilities': ['code', 'programming', 'debug'],
        'priority': 1
    }
}

# Filter to only include models recommended for this system
CATALOG = {k: v for k, v in CATALOG.items() if k in model_aliases}

print('📋 Active Model Catalog (Hardware-Optimized Aliases)')
print('=' * 70)
print('💡 Using model aliases - Foundry automatically selects CPU variants')
print()
for model, info in CATALOG.items():
    caps = ', '.join(info['capabilities'])
    print(f'   • {model}')
    print(f'     Capabilities: {caps}')
    print(f'     Priority: {info["priority"]}')
    print()

# Intent detection rules (regex pattern -> intent label)
INTENT_RULES = [
    (re.compile(r'code|refactor|function|debug|program', re.I), 'code'),
    (re.compile(r'summar|abstract|tl;?dr|brief', re.I), 'summarize'),
    (re.compile(r'classif|categor|label|sentiment', re.I), 'classification'),
    (re.compile(r'explain|teach|describe', re.I), 'general'),
]

def detect_intent(prompt: str) -> str:
    """Detect intent from prompt using regex patterns.
    
    Args:
        prompt: User input text
        
    Returns:
        Intent label: 'code', 'summarize', 'classification', or 'general'
    """
    for pattern, intent in INTENT_RULES:
        if pattern.search(prompt):
            return intent
    return 'general'

def pick_model(intent: str) -> str:
    """Select best model for intent based on capabilities and priority.
    
    Args:
        intent: Detected intent category
        
    Returns:
        Model alias string, or first available model if no match
    """
    candidates = [
        (alias, info['priority']) 
        for alias, info in CATALOG.items() 
        if intent in info['capabilities']
    ]
    
    if candidates:
        # Sort by priority (higher = better)
        candidates.sort(key=lambda x: x[1], reverse=True)
        return candidates[0][0]
    
    # Fallback to first available model
    return list(CATALOG.keys())[0] if CATALOG else None

print('✅ Intent detection and model selection configured')
print('=' * 70)

📋 Active Model Catalog (Hardware-Optimized Aliases)
💡 Using model aliases - Foundry automatically selects CPU variants

   • phi-4-mini
     Capabilities: general, summarize, reasoning
     Priority: 3

   • qwen2.5-0.5b
     Capabilities: classification, fast, general
     Priority: 1

   • phi-3.5-mini
     Capabilities: code, refactor, technical
     Priority: 2

   • qwen2.5-coder-0.5b
     Capabilities: code, programming, debug
     Priority: 1

✅ Intent detection and model selection configured

💡 Using model aliases - Foundry automatically selects CPU variants

   • phi-4-mini
     Capabilities: general, summarize, reasoning
     Priority: 3

   • qwen2.5-0.5b
     Capabilities: classification, fast, general
     Priority: 1

   • phi-3.5-mini
     Capabilities: code, refactor, technical
     Priority: 2

   • qwen2.5-coder-0.5b
     Capabilities: code, programming, debug
     Priority: 1

✅ Intent detection and model selection configured


## 🧪 चरण 5: हेतू शोधण्याची चाचणी करा

पहा की हेतू शोधण्याची प्रणाली विविध प्रकारच्या प्रॉम्प्ट्सचे योग्य वर्गीकरण करते.


In [111]:
# Test intent detection with sample prompts
test_prompts = [
    'Refactor this Python function for better readability',
    'Summarize the key points of this article',
    'Classify this customer feedback as positive or negative',
    'Explain how edge AI differs from cloud AI',
    'Write a function to calculate fibonacci numbers',
    'Give me a brief overview of small language models'
]

print('🧪 Testing Intent Detection')
print('=' * 70)

for prompt in test_prompts:
    intent = detect_intent(prompt)
    model = pick_model(intent)
    print(f'\nPrompt: {prompt[:50]}...')
    print(f'   Intent: {intent:15s} → Model: {model}')

print('\n' + '=' * 70)
print('✅ Intent detection working correctly')

🧪 Testing Intent Detection

Prompt: Refactor this Python function for better readabili...
   Intent: code            → Model: phi-3.5-mini

Prompt: Summarize the key points of this article...
   Intent: summarize       → Model: phi-4-mini

Prompt: Classify this customer feedback as positive or neg...
   Intent: classification  → Model: qwen2.5-0.5b

Prompt: Explain how edge AI differs from cloud AI...
   Intent: general         → Model: phi-4-mini

Prompt: Write a function to calculate fibonacci numbers...
   Intent: code            → Model: phi-3.5-mini

Prompt: Give me a brief overview of small language models...
   Intent: summarize       → Model: phi-4-mini

✅ Intent detection working correctly


## 🚀 चरण 6: रूटिंग फंक्शन लागू करा

मुख्य रूटिंग फंक्शन तयार करा जो:
1. प्रॉम्प्टमधून हेतू ओळखतो
2. सर्वोत्तम मॉडेल निवडतो
3. Foundry Local SDK च्या माध्यमातून विनंती पूर्ण करतो
4. टोकन वापर आणि त्रुटींचे ट्रॅकिंग करतो

**workshop_utils पॅटर्न वापरतो**:
- एक्सपोनेंशियल बॅकऑफसह स्वयंचलित पुनर्प्रयत्न
- OpenAI-सुसंगत API
- टोकन ट्रॅकिंग आणि त्रुटी हाताळणी


In [112]:
import os
from workshop_utils import chat_once

# Fix RETRY_BACKOFF environment variable if it has comments
if 'RETRY_BACKOFF' in os.environ:
    retry_val = os.environ['RETRY_BACKOFF'].strip().split()[0]
    try:
        float(retry_val)
        os.environ['RETRY_BACKOFF'] = retry_val
    except ValueError:
        os.environ['RETRY_BACKOFF'] = '1.0'

def route(prompt: str, max_tokens: int = 200, temperature: float = 0.7):
    """Route prompt to appropriate model based on intent.
    
    Pipeline:
    1. Detect intent using regex patterns
    2. Select best model by capability + priority
    3. Execute via Foundry Local SDK
    
    Args:
        prompt: User input text
        max_tokens: Maximum tokens in response
        temperature: Sampling temperature (0-1)
        
    Returns:
        Dict with: intent, model, output, tokens, usage, error
    """
    intent = detect_intent(prompt)
    model_alias = pick_model(intent)
    
    if not model_alias:
        return {
            'intent': intent,
            'model': None,
            'output': '',
            'tokens': None,
            'usage': {},
            'error': 'No suitable model found'
        }
    
    try:
        # Call Foundry Local via workshop_utils
        text, usage = chat_once(
            model_alias,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            temperature=temperature
        )
        
        # Extract token information
        usage_info = {}
        if usage:
            usage_info['prompt_tokens'] = getattr(usage, 'prompt_tokens', None)
            usage_info['completion_tokens'] = getattr(usage, 'completion_tokens', None)
            usage_info['total_tokens'] = getattr(usage, 'total_tokens', None)
        
        # Estimate if not provided
        if not usage_info.get('total_tokens'):
            est_prompt = len(prompt) // 4
            est_completion = len(text or '') // 4
            usage_info['estimated_tokens'] = est_prompt + est_completion
        
        return {
            'intent': intent,
            'model': model_alias,
            'output': (text or '').strip(),
            'tokens': usage_info.get('total_tokens') or usage_info.get('estimated_tokens'),
            'usage': usage_info,
            'error': None
        }
    
    except Exception as e:
        return {
            'intent': intent,
            'model': model_alias,
            'output': '',
            'tokens': None,
            'usage': {},
            'error': f'{type(e).__name__}: {str(e)}'
        }

print('✅ Routing function ready')
print('   Using Foundry Local SDK via workshop_utils')
print('   Token tracking: Enabled')
print('   Retry logic: Automatic with exponential backoff')

✅ Routing function ready
   Using Foundry Local SDK via workshop_utils
   Token tracking: Enabled
   Retry logic: Automatic with exponential backoff


## 🎯 पायरी 7: रूटिंग चाचण्या चालवा

विविध प्रॉम्प्टसह संपूर्ण रूटिंग प्रणालीची चाचणी घ्या आणि दाखवा:
- स्वयंचलित हेतू ओळख
- बुद्धिमान मॉडेल निवड
- अनेक मॉडेल्ससह रूटिंग आणि मॉडेल्स टिकवून ठेवणे
- टोकन ट्रॅकिंग आणि कार्यक्षमता मेट्रिक्स


In [ ]:
# Test prompts covering all intent categories
test_cases = [
    {
        'prompt': 'Refactor this Python function to make it more efficient and readable',
        'expected_intent': 'code'
    },
    {
        'prompt': 'Summarize the key benefits of using small language models at the edge',
        'expected_intent': 'summarize'
    },
    {
        'prompt': 'Classify this user feedback: The app is slow but the UI looks great',
        'expected_intent': 'classification'
    },
    {
        'prompt': 'Explain the difference between local and cloud inference',
        'expected_intent': 'general'
    },
    {
        'prompt': 'Write a Python function to calculate the Fibonacci sequence',
        'expected_intent': 'code'
    },
    {
        'prompt': 'Give me a brief overview of the Phi model family',
        'expected_intent': 'summarize'
    }
]

print('🎯 Running Intent-Based Routing Tests')
print('=' * 80)

results = []
for i, test in enumerate(test_cases, 1):
    print(f'\n[{i}/{len(test_cases)}] Testing prompt...')
    print(f'Prompt: {test["prompt"]}')
    
    result = route(test['prompt'], max_tokens=150)
    results.append(result)
    
    print(f'   Expected Intent: {test["expected_intent"]}')
    print(f'   Detected Intent: {result["intent"]} {"✅" if result["intent"] == test["expected_intent"] else "⚠️"}')
    print(f'   Selected Model:  {result["model"]}')
    
    if result['error']:
        print(f'   ❌ Error: {result["error"]}')
    else:
        output_preview = result['output'][:100] + '...' if len(result['output']) > 100 else result['output']
        print(f'   ✅ Response: {output_preview}')
        
        tokens = result.get('tokens', 0)
        if tokens:
            usage = result.get('usage', {})
            if 'estimated_tokens' in usage:
                print(f'   📊 Tokens: ~{tokens} (estimated)')
            else:
                print(f'   📊 Tokens: {tokens}')

# Summary statistics
print('\n' + '=' * 80)
print('📊 ROUTING SUMMARY')
print('=' * 80)

success_count = sum(1 for r in results if not r['error'])
total_tokens = sum(r.get('tokens', 0) or 0 for r in results if not r['error'])
intent_accuracy = sum(1 for i, r in enumerate(results) if r['intent'] == test_cases[i]['expected_intent'])

print(f'Total Prompts:        {len(results)}')
print(f'✅ Successful:         {success_count}/{len(results)}')
print(f'❌ Failed:             {len(results) - success_count}')
print(f'🎯 Intent Accuracy:    {intent_accuracy}/{len(results)} ({intent_accuracy/len(results)*100:.1f}%)')
print(f'📊 Total Tokens Used:  {total_tokens}')

# Model usage distribution
print('\n📋 Model Usage Distribution:')
model_counts = {}
for r in results:
    if r['model']:
        model_counts[r['model']] = model_counts.get(r['model'], 0) + 1

for model, count in sorted(model_counts.items(), key=lambda x: x[1], reverse=True):
    percentage = (count / len(results)) * 100
    print(f'   • {model}: {count} requests ({percentage:.1f}%)')

if success_count == len(results):
    print('\n🎉 All routing tests passed successfully!')
else:
    print(f'\n⚠️  {len(results) - success_count} test(s) failed')
    print('   Check Foundry Local service: foundry service status')
    print('   Verify models loaded: foundry model ls')

print('=' * 80)

🎯 Running Intent-Based Routing Tests

[1/6] Testing prompt...
Prompt: Refactor this Python function to make it more efficient and readable


   Expected Intent: code
   Detected Intent: code ✅
   Selected Model:  phi-3.5-mini
   ✅ Response: To refactor a Python function for efficiency and readability, I would need to see the specific funct...
   📊 Tokens: ~158 (estimated)

[2/6] Testing prompt...
Prompt: Summarize the key benefits of using small language models at the edge
   Expected Intent: summarize
   Detected Intent: summarize ✅
   Selected Model:  phi-4-mini
   ❌ Error: APIConnectionError: Connection error.

[3/6] Testing prompt...
Prompt: Classify this user feedback: The app is slow but the UI looks great
   Expected Intent: classification
   Detected Intent: classification ✅
   Selected Model:  qwen2.5-0.5b
   ❌ Error: APIConnectionError: Connection error.

[4/6] Testing prompt...
Prompt: Explain the difference between local and cloud inference
   Expected Intent: general
   Detected Intent: general ✅
   Selected Model:  phi-4-mini
   ❌ Error: APIConnectionError: Connection error.

[5/6] Testing prompt...
Prompt: Wr

## 🔧 चरण 8: परस्पर चाचणी

तुमचे स्वतःचे प्रॉम्प्ट वापरून राउटिंग प्रणाली कशी कार्य करते ते पाहा!


In [ ]:
# Interactive testing - modify the prompt and run this cell
custom_prompt = "Explain how model quantization reduces memory usage"

print('🎯 Interactive Routing Test')
print('=' * 80)
print(f'Your prompt: {custom_prompt}')
print()

result = route(custom_prompt, max_tokens=200)

print(f'Detected Intent: {result["intent"]}')
print(f'Selected Model:  {result["model"]}')
print()

if result['error']:
    print(f'❌ Error: {result["error"]}')
else:
    print('✅ Response:')
    print('-' * 80)
    print(result['output'])
    print('-' * 80)
    
    if result['tokens']:
        print(f'\n📊 Tokens used: {result["tokens"]}')

print('\n💡 Try different prompts to test routing behavior!')

🎯 Interactive Routing Test
Your prompt: Explain how model quantization reduces memory usage

Detected Intent: general
Selected Model:  phi-4-mini

✅ Response:
--------------------------------------------------------------------------------
Model quantization is a technique used to reduce the memory footprint of a machine learning model, particularly deep learning models. It works by converting the high-precision weights of a neural network, typically represented as 32-bit floating-point numbers, into lower-precision representations, such as 8-bit integers or even binary values.


The primary reason for quantization is to decrease the amount of memory required to store the model's parameters. Since floating-point numbers take up more space than integers, by quantizing the weights, we can significantly reduce the model's size. This reduction in size not only saves memory but also can lead to faster computation during inference, as integer operations are generally faster than floating-poi

## 📊 चरण 9: कार्यक्षमता विश्लेषण

रूटिंग प्रणालीची कार्यक्षमता आणि मॉडेलचा उपयोग विश्लेषित करा.


In [ ]:
import time

# Performance benchmark
benchmark_prompts = [
    'Write a hello world function',
    'Summarize: AI at the edge is powerful',
    'Classify: Good product',
    'Explain edge computing'
]

print('⚡ Performance Benchmark')
print('=' * 80)

timings = []
for prompt in benchmark_prompts:
    start = time.time()
    result = route(prompt, max_tokens=50)
    duration = time.time() - start
    timings.append(duration)
    
    print(f'\nPrompt: {prompt[:40]}...')
    print(f'   Model: {result["model"]}')
    print(f'   Time: {duration:.2f}s')
    if result.get('tokens'):
        print(f'   Tokens: {result["tokens"]}')

print('\n' + '=' * 80)
print('📊 Performance Statistics:')
print(f'   Average response time: {sum(timings)/len(timings):.2f}s')
print(f'   Fastest response:      {min(timings):.2f}s')
print(f'   Slowest response:      {max(timings):.2f}s')
print('\n💡 Note: First request may be slower due to model initialization')
print('=' * 80)

⚡ Performance Benchmark

Prompt: Write a hello world function...
   Model: phi-3.5-mini
   Time: 3.31s
   Tokens: 60

Prompt: Write a hello world function...
   Model: phi-3.5-mini
   Time: 3.31s
   Tokens: 60

Prompt: Summarize: AI at the edge is powerful...
   Model: phi-4-mini
   Time: 49.67s
   Tokens: 84

Prompt: Summarize: AI at the edge is powerful...
   Model: phi-4-mini
   Time: 49.67s
   Tokens: 84

Prompt: Classify: Good product...
   Model: qwen2.5-0.5b
   Time: 7.21s
   Tokens: 69

Prompt: Classify: Good product...
   Model: qwen2.5-0.5b
   Time: 7.21s
   Tokens: 69

Prompt: Explain edge computing...
   Model: phi-4-mini
   Time: 49.67s
   Tokens: 72

📊 Performance Statistics:
   Average response time: 27.46s
   Fastest response:      3.31s
   Slowest response:      49.67s

💡 Note: First request may be slower due to model initialization

Prompt: Explain edge computing...
   Model: phi-4-mini
   Time: 49.67s
   Tokens: 72

📊 Performance Statistics:
   Average response time:

## 🎓 मुख्य मुद्दे आणि पुढील पावले

### ✅ तुम्ही काय शिकलात

1. **इंटेंट-आधारित रूटिंग**: प्रॉम्प्ट्स आपोआप वर्गीकृत करा आणि विशेष मॉडेल्सकडे रूट करा  
2. **मेमरी-अवेयर निवड**: उपलब्ध सिस्टम RAM च्या आधारे CPU मॉडेल्स निवडा  
3. **मल्टी-मॉडेल रिटेन्शन**: `--retain true` वापरून अनेक मॉडेल्स लोड ठेवा  
4. **प्रॉडक्शन पॅटर्न्स**: रिट्राय लॉजिक, एरर हँडलिंग, आणि टोकन ट्रॅकिंग  
5. **CPU ऑप्टिमायझेशन**: GPU शिवाय कार्यक्षमतेने डिप्लॉय करा  

### 🚀 प्रयोग कल्पना

1. **कस्टम इंटेंट्स जोडा**:  
   ```python
   INTENT_RULES.append(
       (re.compile(r'translate|convert', re.I), 'translation')
   )
   ```
  
2. **अतिरिक्त मॉडेल्स लोड करा**:  
   ```bash
   foundry model run llama-3.2-1b-cpu --retain true
   ```
  
3. **मॉडेल निवड ट्यून करा**:  
   - CATALOG मध्ये प्राधान्य मूल्ये समायोजित करा  
   - अधिक क्षमता टॅग्स जोडा  
   - फॉलबॅक स्ट्रॅटेजी अंमलात आणा  

4. **कामगिरीचे निरीक्षण करा**:  
   ```python
   import psutil
   print(f"Memory: {psutil.virtual_memory().percent}%")
   ```
  

### 📚 अतिरिक्त संसाधने

- **Foundry Local SDK**: https://github.com/microsoft/Foundry-Local  
- **वर्कशॉप नमुने**: ../samples/  
- **एज AI कोर्स**: ../../Module08/  

### 💡 सर्वोत्तम पद्धती

✅ क्रॉस-प्लॅटफॉर्म सुसंगततेसाठी CPU मॉडेल्स वापरा  
✅ अनेक मॉडेल्स लोड करण्यापूर्वी सिस्टम मेमरी नेहमी तपासा  
✅ रूटिंग परिस्थितीसाठी `--retain true` वापरा  
✅ योग्य एरर हँडलिंग आणि रिट्राय अंमलात आणा  
✅ खर्च/कामगिरी ऑप्टिमायझेशनसाठी टोकन वापर ट्रॅक करा  

---

**🎉 अभिनंदन!** तुम्ही Foundry Local SDK वापरून CPU-ऑप्टिमाइझ्ड मॉडेल्ससह उत्पादन-तयार इंटेंट-आधारित मॉडेल राउटर तयार केला आहे!



---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) चा वापर करून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी कृपया लक्षात ठेवा की स्वयंचलित भाषांतरे त्रुटी किंवा अचूकतेच्या अभावाने युक्त असू शकतात. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर करून उद्भवलेल्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
